<a href="https://colab.research.google.com/github/BlackCurrantDS/ATiML-Project/blob/master/Getting_Data_From_Guten_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### *This Notebook Unzip the data downloaded from the site and put the metadata and html book pages into single dataframe by matching on bookid/File no. Also does a little EDA on meta_data and remove html tags from the the combined dataframe books and exports the complete data to csv for further analysis.*

In [ ]:
#Unzip the Overall data donwloaded from the site
!unzip Gutenberg_English_Fiction_1k.zip -d books_set


Archive:  Gutenberg_English_Fiction_1k.zip
   creating: books_set/Gutenberg_English_Fiction_1k/
   creating: books_set/Gutenberg_English_Fiction_1k/Gutenberg_19th_century_English_Fiction/
  inflating: books_set/Gutenberg_English_Fiction_1k/Gutenberg_19th_century_English_Fiction/pg10067-content.html  
  inflating: books_set/Gutenberg_English_Fiction_1k/Gutenberg_19th_century_English_Fiction/pg1032-content.html  
  inflating: books_set/Gutenberg_English_Fiction_1k/Gutenberg_19th_century_English_Fiction/pg10379-content.html  
  inflating: books_set/Gutenberg_English_Fiction_1k/Gutenberg_19th_century_English_Fiction/pg10473-content.html  
  inflating: books_set/Gutenberg_English_Fiction_1k/Gutenberg_19th_century_English_Fiction/pg105JaneAusPersuatn-content.html  
  inflating: books_set/Gutenberg_English_Fiction_1k/Gutenberg_19th_century_English_Fiction/pg10812-content.html  
  inflating: books_set/Gutenberg_English_Fiction_1k/Gutenberg_19th_century_English_Fiction/pg10826-content.html  
  

In [ ]:
#Reading the meta data file, it has label and authorname.book name etc
import pandas as pd

data = []

%time

import os
file = os.path.abspath('books_set/Gutenberg_English_Fiction_1k/master996.csv')

with open(file, "rb") as metafile:
    # decode it as 'utf-8', remove any special characters, and split it on the comma (or deliminator)
    header = metafile.readline().decode('utf-8').replace('\r\n', '').split(';')
    # read the rest of the data
    for line in metafile:
        row = line.decode('utf-8', errors='ignore').replace('\r\n', '').split(';')
        data.append(row)

# save the data as a dataframe
meta_data = pd.DataFrame(data=data, columns = header)
meta_data.head(5)

In [ ]:
#Extracting the bookid into Dataframe column FileNo, book id pg10067- FileNo - 10067

meta_data['FileNo'] = meta_data['book_id'].str.replace(r'\D+', '').astype(int)

#Checkign the values

meta_data.sort_values(by=['FileNo'], inplace=True)
meta_data

,Book_Name,book_id,guten_genre,Author_Name,FileNo
961,Alice's Adventures in Wonderland,pg11CarolAlice.epub,Literary,Carroll| Lewis,11
962,Through the Looking-Glass,pg12CarolGlassLook.epub,Literary,Carroll| Lewis,12
995,A Tale of Two Cities,pg98DickensTaleCities.epub,Literary,Dickens| Charles,98
958,Persuasion,pg105JaneAusPersuatn.epub,Literary,Austen| Jane,105
959,Return of Sherlock Holmes,pg108DoyleReturnSherlk.epub,Detective and Mystery,Connan| Doyle,108
...,...,...,...,...,...
863,Kate Vernon: A Tale. Vol. 2 (of 3),pg53356.epub,Literary,Alexander| Mrs.,53356
864,The Touch of Abner,pg53604.epub,Literary,Cody| H. A.,53604
865,Kate Vernon: A Tale. Vol. 3 (of 3),pg53668.epub,Literary,Alexander| Mrs.,53668
964,Mardi: and A Voyage Thither I,pg13720HermanVoyage1.epub,Sea and Adventure,Melville| Herman,137201


In [ ]:
#Checking memory usage in detail with distribution of data
meta_data.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 996 entries, 961 to 965
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Book_Name    996 non-null    object
 1   book_id      996 non-null    object
 2   guten_genre  996 non-null    object
 3   Author_Name  996 non-null    object
 4   FileNo       996 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 302.7 KB


In [ ]:
#Chcking the shape of the dataframe

meta_data.shape


(996, 5)

In [ ]:
#Checking the count of the values in dataframe, should match wiht the file

meta_data.count()

Book_Name      996
book_id        996
guten_genre    996
Author_Name    996
FileNo         996
dtype: int64

In [ ]:
#Checking Null Values

meta_data.info() 

#There are no null values

<class 'pandas.core.frame.DataFrame'>
Int64Index: 996 entries, 961 to 965
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Book_Name    996 non-null    object
 1   book_id      996 non-null    object
 2   guten_genre  996 non-null    object
 3   Author_Name  996 non-null    object
 4   FileNo       996 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 46.7+ KB


In [ ]:
#Getting list of gernes

print(meta_data.guten_genre.value_counts().unique())

[794 111  36  18   6   5   2]


In [ ]:
#Checking distribution of Gerne columns

meta_data.guten_genre.describe()

count          996
unique           9
top       Literary
freq           794
Name: guten_genre, dtype: object

In [ ]:
#Getting distribution of authors

meta_data.Author_Name.describe()

""" List is quite long"""

#Getting list of authors

# print(data.Author_Name.unique())

' List is quite long'

Getting HTML books from the folder

In [ ]:
import glob

# Getting all files and its names into dict and then converting to dataframe"""

Content = {}

#Its placed in Books folder at local , need to change it later 
%timeit

files = glob.glob("books_set//Gutenberg_English_Fiction_1k//Gutenberg_19th_century_English_Fiction//*.html")
for f in files:
    with open(f,mode ='r', encoding = 'utf8') as myfile:
        Content[f]=myfile.read().replace("<br>", '\n')


In [ ]:
#Putting it in dataframe
df = pd.DataFrame.from_dict(Content,orient='index').reset_index()

#Renaming column names
df.columns = ['File_Name', 'Data']
print(df.head(5))
print(df['File_Name'][0])
df['File_Name'] = df['File_Name'].str.replace("books_set//Gutenberg_English_Fiction_1k//Gutenberg_19th_century_English_Fiction/", "l")
print(df['File_Name'][0])
#Adding FilNo column to join
df['FileNo'] = df['File_Name'].str.replace(r'\D+', '').astype(int)
left = df
right = meta_data
result = pd.merge(left, right , how='inner', on=['FileNo', 'FileNo'])

#Checking resulting dataframe
result.info()

#Dropping unnecessery columns

books=result.drop(['File_Name', 'book_id'], axis=1)
books.head(5)

                                           File_Name                                               Data
0  books_set//Gutenberg_English_Fiction_1k//Guten...  <p>This etext was produced by Gardner Buchanan...
1  books_set//Gutenberg_English_Fiction_1k//Guten...  <p>IF it had not been for Bunsey, the novelist...
2  books_set//Gutenberg_English_Fiction_1k//Guten...  <p>This book has its title from that dashing s...
3  books_set//Gutenberg_English_Fiction_1k//Guten...  <p>E-text prepared by Lionel Sear\n<p>Joseph W...
4  books_set//Gutenberg_English_Fiction_1k//Guten...  <p>Edition: 10\n<p>The Prose of Alfred Lichten...
books_set//Gutenberg_English_Fiction_1k//Gutenberg_19th_century_English_Fiction/pg14490-content.html
lpg14490-content.html
<class 'pandas.core.frame.DataFrame'>
Int64Index: 996 entries, 0 to 995
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   File_Name    996 non-null    object
 1   Data         996 non-

,Data,FileNo,Book_Name,guten_genre,Author_Name
0,<p>This etext was produced by Gardner Buchanan...,14490,A Daughter of To-Day,Literary,Duncan| Sara Jeannette
1,"<p>IF it had not been for Bunsey, the novelist...",20661,The Romance of an Old Fool,Love and Romance,Field| Roswell Martin
2,<p>This book has its title from that dashing s...,6410,Once Aboard the Lugger-- The History of George...,Humorous and Wit and Satire,Hutchinson| A. S. M.
3,<p>Edition: 10\n<p>The Prose of Alfred Lichten...,5663,The Prose of Alfred Lichtenstein,Literary,Lichtenstein| Alfred
4,<p>Decorative graphic\n<p>This edition first p...,1193,The Coxon Fund,Literary,James| Henry


Checking label counts of all books

In [ ]:
from collections import Counter
print('Dataset shape %s' % Counter(books['guten_genre']))

Dataset shape Counter({'Literary': 794, 'Detective and Mystery': 111, 'Sea and Adventure': 36, 'Love and Romance': 18, 'Western Stories': 18, 'Humorous and Wit and Satire': 6, 'Ghost and Horror': 6, 'Christmas Stories': 5, 'Allegories': 2})


Cleaning HTML

In [ ]:
from bs4 import BeautifulSoup
books['Data']= [BeautifulSoup(text).get_text() for text in books['Data'] ]
books.head()

,Data,FileNo,Book_Name,guten_genre,Author_Name
0,This etext was produced by Gardner Buchanan.\n...,14490,A Daughter of To-Day,Literary,Duncan| Sara Jeannette
1,"IF it had not been for Bunsey, the novelist, I...",20661,The Romance of an Old Fool,Love and Romance,Field| Roswell Martin
2,This book has its title from that dashing sent...,6410,Once Aboard the Lugger-- The History of George...,Humorous and Wit and Satire,Hutchinson| A. S. M.
3,Edition: 10\nThe Prose of Alfred Lichtenstein ...,5663,The Prose of Alfred Lichtenstein,Literary,Lichtenstein| Alfred
4,Decorative graphic\nThis edition first publish...,1193,The Coxon Fund,Literary,James| Henry


In [ ]:
books.rename(columns = {'Data': 'data', 'guten_genre': 'genre'} , inplace = True)

Exporting Data to csv file for further EDA...

In [ ]:
books.to_csv('books.csv')

In [ ]:
meta_data.to_csv("books_meta_data.csv")

In [ ]:
new_books = books[['FileNo','data', 'genre', 'Author_Name', 'Book_Name']]
new_books.to_csv("books_base_line.csv")